In [ ]:
# Step 1: Set up GPU in Google Colab
import tensorflow as tf

# Check if GPU is available
print("GPU Available:", tf.config.list_physical_devices('GPU'))

# Ensure TensorFlow is using the GPU
if tf.config.list_physical_devices('GPU'):
    print("Using GPU for training")
else:
    print("GPU not found. Using CPU instead.")

# Step 2: Unzip the uploaded archive.zip file
import zipfile
import os

with zipfile.ZipFile('archive.zip', 'r') as zip_ref:
    zip_ref.extractall('emotion_dataset')

# Define paths to the train and test folders
train_dir = 'emotion_dataset/train'
test_dir = 'emotion_dataset/test'

# Step 3: Use ImageDataGenerator to preprocess and augment the data
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,  # Normalize pixel values to [0, 1]
    rotation_range=20,     # Randomly rotate images
    width_shift_range=0.2, # Randomly shift images horizontally
    height_shift_range=0.2, # Randomly shift images vertically
    shear_range=0.2,       # Apply shear transformations
    zoom_range=0.2,        # Randomly zoom images
    horizontal_flip=True,  # Randomly flip images horizontally
    fill_mode='nearest'    # Fill missing pixels with the nearest value
)

test_datagen = ImageDataGenerator(rescale=1.0 / 255.0)  # Only rescale for test data

# Create data generators for training and testing
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(48, 48),  # Resize images to 48x48
    batch_size=64,
    color_mode='grayscale', # Use grayscale images
    class_mode='categorical' # Use categorical labels
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(48, 48),
    batch_size=64,
    color_mode='grayscale',
    class_mode='categorical',
    shuffle=False  # Do not shuffle test data
)

# Step 4: Build the CNN model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(7, activation='softmax')  # 7 classes for 7 emotions
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Step 5: Train the model using GPU
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=20,
    validation_data=test_generator,
    validation_steps=test_generator.samples // test_generator.batch_size
)

# Step 6: Evaluate the model
test_loss, test_acc = model.evaluate(test_generator)
print(f'Test Accuracy: {test_acc:.4f}')

# Generate predictions
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

y_pred = model.predict(test_generator)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = test_generator.classes

# Print classification report
print("Classification Report:")
print(classification_report(y_true_classes, y_pred_classes, target_names=train_generator.class_indices.keys()))

# Print confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y_true_classes, y_pred_classes))

# Step 7: Save the model
model.save('emotion_detection_model.h5')

# Step 8: Download the model to your local machine
from google.colab import files
files.download('emotion_detection_model.h5')

GPU Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Using GPU for training
Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


448/448 ━━━━━━━━━━━━━━━━━━━━ 31s 57ms/step - accuracy: 0.2386 - loss: 1.8279 - val_accuracy: 0.2633 - val_loss: 1.7688
Epoch 2/20
  1/448 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.2344 - loss: 1.7876

...

448/448 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.2344 - loss: 1.7876 - val_accuracy: 0.2674 - val_loss: 1.7684
Epoch 3/20
448/448 ━━━━━━━━━━━━━━━━━━━━ 33s 50ms/step - accuracy: 0.2562 - loss: 1.7774 - val_accuracy: 0.2956 - val_loss: 1.7010
Epoch 4/20
448/448 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.1875 - loss: 1.8035 - val_accuracy: 0.2970 - val_loss: 1.7009
Epoch 5/20
448/448 ━━━━━━━━━━━━━━━━━━━━ 23s 50ms/step - accuracy: 0.2805 - loss: 1.7403 - val_accuracy: 0.3475 - val_loss: 1.5996
Epoch 6/20
448/448 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.2969 - loss: 1.5720 - val_accuracy: 0.3426 - val_loss: 1.5994
Epoch 7/20
448/448 ━━━━━━━━━━━━━━━━━━━━ 41s 54ms/step - accuracy: 0.3208 - loss: 1.6853 - val_accuracy: 0.4270 - val_loss: 1.4900
Epoch 8/20
448/448 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.3438 - loss: 1.5935 - val_accuracy: 0.4191 - val_loss: 1.4897
Epoch 9/20
448/448 ━━━━━━━━━━━━━━━━━━━━ 23s 50ms/step - accuracy: 0.3728 - loss: 1.5890 - val_accuracy: 0.460

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Classification Report:
              precision    recall  f1-score   support

       angry       0.38      0.49      0.43       958
   disgusted       0.00      0.00      0.00       111
     fearful       0.33      0.08      0.13      1024
       happy       0.73      0.81      0.77      1774
     neutral       0.47      0.56      0.51      1233
         sad       0.45      0.36      0.40      1247
   surprised       0.53      0.81      0.64       831

    accuracy                           0.53      7178
   macro avg       0.41      0.44      0.41      7178
weighted avg       0.50      0.53      0.50      7178

Confusion Matrix:
[[ 472    0   37   96  161  114   78]
 [  65    0    2   12    8   15    9]
 [ 206    0   81   95  181  200  261]
 [  81    0   29 1439   82   60   83]
 [ 150    0   28  129  688  150   88]
 [ 227    0   44  150  306  452   68]
 [  35    0   27   46   35   18  670]]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')